In [2]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [4]:

load_dotenv()
KEY = os.getenv("OPENAI_API_KEY") 
llm = ChatOpenAI(openai_api_key= KEY, model_name= "gpt-3.5-turbo", temperature ="0.5")

e:\GPT BootCamp\OPENAI\MCQGenerator\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [6]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

# tonme: Difficulty level of the question : simple| intermediate| Hard

In [7]:
prompt_template = PromptTemplate(
    input_variables = ["text","number","subject","tone","response_json"],
    template = TEMPLATE
)

In [8]:
chain = LLMChain(llm = llm, prompt = prompt_template, output_key = "quiz", verbose= True)

In [9]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [10]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [11]:
evalChain = LLMChain(llm=llm, prompt= quiz_evaluation_prompt, output_key= "review", verbose= True)

In [12]:
seqChain = SequentialChain(chains = [chain, evalChain], input_variables=["text","number","subject","tone","response_json"],
                           output_variables=["quiz", "review"], verbose = True)

In [58]:
filePath = r"E:\GPT BootCamp\OPENAI\MCQGenerator\Excretion.txt"

with open(filePath, 'r') as file:
    TEXT = file.read()

In [ ]:
print(TEXT)

In [59]:
json.dumps(RESPONSE_JSON)

NUMBER=10
SUBJECT="Biology"
TONE="hard"

In [60]:
with get_openai_callback() as cb:
    response=seqChain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Osmoregulation: 
Regulation of solute and water concentration, maintenance of constant osmotic pressure. Part of homeostasis.
Osmoregulators: regulate osmotic pressure independent from environment
Osmoconformers: regulate osmotic pressure according to environment (jellyfish, mussels. Some can work better with changing salt concentrations around them)



Salt glands
•	Excrete hypertonic substances with help of NaCl (costs ATP)
•	Located in skull of some reptiles and most water birds



Nitrogenous waste
Are broken down to glucose and ammonia. 
Excretion of 
•	Ammonium: most aquatic animals and bony fishes. Highly poisonous, excretion (both active and passive) via kidneys and gills requires lots of water
•	Urea: Mammals, shark, some bony fishes, most amphibians. Less toxic than ammonium thus less water required for excretion. CO2 binds to ammonium in liver (energy-demanding) creati

In [61]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2493
Prompt Tokens:1736
Completion Tokens:757
Total Cost:0.004118


In [62]:
response

{'text': 'Osmoregulation: \nRegulation of solute and water concentration, maintenance of constant osmotic pressure. Part of homeostasis.\nOsmoregulators: regulate osmotic pressure independent from environment\nOsmoconformers: regulate osmotic pressure according to environment (jellyfish, mussels. Some can work better with changing salt concentrations around them)\n\n\n\nSalt glands\n•\tExcrete hypertonic substances with help of NaCl (costs ATP)\n•\tLocated in skull of some reptiles and most water birds\n\n\n\nNitrogenous waste\nAre broken down to glucose and ammonia. \nExcretion of \n•\tAmmonium: most aquatic animals and bony fishes. Highly poisonous, excretion (both active and passive) via kidneys and gills requires lots of water\n•\tUrea: Mammals, shark, some bony fishes, most amphibians. Less toxic than ammonium thus less water required for excretion. CO2 binds to ammonium in liver (energy-demanding) creating urea.\n•\tUric acid: Birds, insects, many reptiles, land snails. Relativel

In [63]:
quiz=response.get("quiz")
json.loads(quiz)

{'1': {'mcq': 'What is the main function of osmoregulation?',
  'options': {'a': 'Regulation of solute concentration only',
   'b': 'Maintenance of constant osmotic pressure only',
   'c': 'Regulation of solute and water concentration, maintenance of constant osmotic pressure',
   'd': 'Regulation of water concentration only'},
  'correct': 'c'},
 '2': {'mcq': 'Which of the following organisms are osmoregulators?',
  'options': {'a': 'Jellyfish',
   'b': 'Mussels',
   'c': 'Both Jellyfish and Mussels',
   'd': 'None of the above'},
  'correct': 'd'},
 '3': {'mcq': 'Where are salt glands located in some reptiles and most water birds?',
  'options': {'a': 'In the tail',
   'b': 'In the abdomen',
   'c': 'In the skull',
   'd': 'In the limbs'},
  'correct': 'c'},
 '4': {'mcq': 'Which nitrogenous waste is excreted by most aquatic animals and bony fishes?',
  'options': {'a': 'Ammonium', 'b': 'Urea', 'c': 'Uric acid', 'd': 'Glucose'},
  'correct': 'a'},
 '5': {'mcq': 'What is the functional